In [1]:
import pandas as pd

# Load the CSV files to inspect their contents
clickup_path = 'ClickUp - clickup.csv'
float_path = 'Float - allocations.csv'

# Reading the CSV files
clickup_df = pd.read_csv(clickup_path)
float_df = pd.read_csv(float_path)

# Displaying the first few rows of each dataset
clickup_df.head(), float_df.head()

(     Client              Project                Name    Task        Date  \
 0  Client 1  Website Development  Isabella Rodriguez  Design  2023-07-03   
 1  Client 1  Website Development  Isabella Rodriguez  Design  2023-07-04   
 2  Client 1  Website Development  Isabella Rodriguez  Design  2023-07-05   
 3  Client 1  Website Development  Isabella Rodriguez  Design  2023-07-06   
 4  Client 1  Website Development  Isabella Rodriguez  Design  2023-07-07   
 
    Hours                                        Note Billable  
 0    6.5                     Refined design elements      Yes  
 1    6.5             Drafted initial design concepts      Yes  
 2    6.0             Drafted initial design concepts      Yes  
 3    7.0  Made revisions to design based on feedback      Yes  
 4    7.0  Made revisions to design based on feedback      Yes  ,
      Client              Project                Role                Name  \
 0  Client 1  Website Development    Product Designer  Isabella Rodr

The next step is to transform the data into fact and dimension tables, ensuring data cleanliness and preparing for ETL implementation. Let's proceed with cleaning and transforming the data

In [ ]:
# Cleaning and transforming the data

In [2]:
# Removing any leading/trailing spaces from column names and entries
clickup_df.columns = clickup_df.columns.str.strip()
float_df.columns = float_df.columns.str.strip()

In [3]:
clickup_df = clickup_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
float_df = float_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [5]:
# Checking for any missing values in both datasets
clickup_missing = clickup_df.isnull().sum()
float_missing = float_df.isnull().sum()

clickup_missing, float_missing

(Client      0
 Project     0
 Name        0
 Task        0
 Date        0
 Hours       0
 Note        0
 Billable    0
 dtype: int64,
 Client             0
 Project            0
 Role               0
 Name               0
 Task               0
 Start Date         0
 End Date           0
 Estimated Hours    0
 dtype: int64)

In [6]:
# Checking for duplicates in both datasets
clickup_duplicates = clickup_df.duplicated().sum()
float_duplicates = float_df.duplicated().sum()

clickup_duplicates, float_duplicates

(0, 0)

The data from both datasets is clean with no missing values or duplicates. 

The next step is to:

Create Fact and Dimension Tables from the cleaned data.
Implement the ETL Process to load the data into these tables.

Dimension Tables:

Dim_TeamMember: From the Name column (common across both datasets).
Dim_Project: From the Project column.
Dim_Role: From the Role column (Float dataset only).
Dim_Date: Extracted from the Date, Start Date, and End Date columns for time-based analysis.


In [7]:
# Creating Dimension Tables

# Dim_TeamMember: unique team member names from both datasets
dim_team_member = pd.DataFrame({
    'TeamMemberID': range(1, clickup_df['Name'].nunique() + 1),
    'TeamMemberName': clickup_df['Name'].unique()
})

dim_team_member.head()


,TeamMemberID,TeamMemberName
0,1,Isabella Rodriguez
1,2,John Smith
2,3,Liu Wei
3,4,Emily Patel
4,5,Ali Khan


In [8]:
# Dim_Project: unique projects from both datasets
dim_project = pd.DataFrame({
    'ProjectID': range(1, clickup_df['Project'].nunique() + 1),
    'ProjectName': clickup_df['Project'].unique()
})
dim_project.head()

,ProjectID,ProjectName
0,1,Website Development
1,2,Brand Guideline
2,3,Book Localization to Ukraine


In [9]:
# Dim_Role: unique roles from the Float dataset
dim_role = pd.DataFrame({
    'RoleID': range(1, float_df['Role'].nunique() + 1),
    'RoleName': float_df['Role'].unique()
})
dim_role.head()

,RoleID,RoleName
0,1,Product Designer
1,2,Design Manager
2,3,Front End Engineer
3,4,QA Engineer
4,5,Project Manager


In [10]:
# Parsing dates for the Dim_Date table from the ClickUp and Float datasets
dates = pd.concat([clickup_df['Date'], float_df['Start Date'], float_df['End Date']]).unique()
dim_date = pd.DataFrame({
    'DateID': range(1, len(dates) + 1),
    'Date': pd.to_datetime(dates)
})
dim_date.head()

,DateID,Date
0,1,2023-07-03
1,2,2023-07-04
2,3,2023-07-05
3,4,2023-07-06
4,5,2023-07-07



Fact Tables:

Fact_Allocations: Will store the Name, Project, Role, Task, Start Date, End Date, and Estimated Hours from the Float dataset.
Fact_TimeTracking: Will store the Name, Project, Task, Date, Hours, and Billable from the ClickUp dataset.

In [11]:
# Merging datasets with dimension tables to create fact tables

# Fact_Allocations: linking the Float dataset to dimensions
fact_allocations = float_df.merge(dim_team_member, left_on='Name', right_on='TeamMemberName') \
                           .merge(dim_project, left_on='Project', right_on='ProjectName') \
                           .merge(dim_role, left_on='Role', right_on='RoleName')



In [12]:
fact_allocations = fact_allocations[['Client','TeamMemberID', 'ProjectID', 'RoleID', 'Task', 
                                     'Start Date', 'End Date', 'Estimated Hours']]
fact_allocations.head()

,Client,TeamMemberID,ProjectID,RoleID,Task,Start Date,End Date,Estimated Hours
0,Client 1,1,1,1,Design,2023-07-03,2023-07-24,112
1,Client 1,2,1,2,Design,2023-07-03,2023-07-24,24
2,Client 1,2,2,2,Design,2023-07-03,2023-07-24,32
3,Client 2,2,3,2,Design,2023-07-24,2023-08-28,52
4,Client 1,3,1,3,Development,2023-07-31,2023-08-28,189


In [13]:

# Fact_TimeTracking: linking the ClickUp dataset to dimensions
fact_time_tracking = clickup_df.merge(dim_team_member, left_on='Name', right_on='TeamMemberName') \
                               .merge(dim_project, left_on='Project', right_on='ProjectName') \
                               #.merge(dim_client, left_on='Client', right_on='Client')

In [14]:
fact_time_tracking = fact_time_tracking[['Client','TeamMemberID', 'ProjectID', 'Date', 'Task', 'Hours', 'Note','Billable']]

# Displaying the first few rows of both fact tables
fact_time_tracking.head()


,Client,TeamMemberID,ProjectID,Date,Task,Hours,Note,Billable
0,Client 1,1,1,2023-07-03,Design,6.5,Refined design elements,Yes
1,Client 1,1,1,2023-07-04,Design,6.5,Drafted initial design concepts,Yes
2,Client 1,1,1,2023-07-05,Design,6.0,Drafted initial design concepts,Yes
3,Client 1,1,1,2023-07-06,Design,7.0,Made revisions to design based on feedback,Yes
4,Client 1,1,1,2023-07-07,Design,7.0,Made revisions to design based on feedback,Yes


Next Step is to Load the following dataframes into BigQuery:

dim_project
dim_role
dim_team_member
fact_allocations
fact_time_tracking

Note: A new dataset will be created on BigQuery named 'clickup_float_allocations'

In [15]:
from google.oauth2 import service_account
from pandas_gbq import to_gbq


service_account_path = "bq_creds.json"

credentials = service_account.Credentials.from_service_account_file(
    service_account_path,
)


In [16]:
project_id = 'composite-store-123456'
dataset_id = 'clickup_float_allocations'

table_names = ["dim_project", "dim_role", "dim_team_member", "fact_allocations", "fact_time_tracking"]


In [18]:
dataframes = [dim_project, dim_role, dim_team_member, fact_allocations, fact_time_tracking]  # Replace with your DataFrames

for table_name, df in zip(table_names, dataframes):
    df.to_gbq(destination_table=f'{project_id}.{dataset_id}.{table_name}',
             project_id=project_id,
             if_exists='append',
             credentials=credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 7695.97it/s]
